# Контекст

загрузка готовой модели Helsinki для перевода eng-rus

# Импотры

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from transformers import MarianMTModel, MarianTokenizer

"import wandb\nwandb.init(mode='disabled')"

# Чтение файла

In [2]:
with open('rus.txt') as file:
    lines = file.readlines()

lines = [line.replace('\n', '').lower().split('\t')[:-1] for line in lines]
lines[:10]

[['go.', 'марш!'],
 ['go.', 'иди.'],
 ['go.', 'идите.'],
 ['hi.', 'здравствуйте.'],
 ['hi.', 'привет!'],
 ['hi.', 'хай.'],
 ['hi.', 'здрасте.'],
 ['hi.', 'здоро́во!'],
 ['hi.', 'приветик!'],
 ['run!', 'беги!']]

In [3]:
data = []
for english_sentence, russian_sentence in lines:
    data.append({'english_tokens': word_tokenize(english_sentence, language = 'english'),
                 'russian_tokens': word_tokenize(russian_sentence, language = 'russian')})
data[:10]

[{'english_tokens': ['go', '.'], 'russian_tokens': ['марш', '!']},
 {'english_tokens': ['go', '.'], 'russian_tokens': ['иди', '.']},
 {'english_tokens': ['go', '.'], 'russian_tokens': ['идите', '.']},
 {'english_tokens': ['hi', '.'], 'russian_tokens': ['здравствуйте', '.']},
 {'english_tokens': ['hi', '.'], 'russian_tokens': ['привет', '!']},
 {'english_tokens': ['hi', '.'], 'russian_tokens': ['хай', '.']},
 {'english_tokens': ['hi', '.'], 'russian_tokens': ['здрасте', '.']},
 {'english_tokens': ['hi', '.'], 'russian_tokens': ['здоро́во', '!']},
 {'english_tokens': ['hi', '.'], 'russian_tokens': ['приветик', '!']},
 {'english_tokens': ['run', '!'], 'russian_tokens': ['беги', '!']}]

In [4]:
data_to_pd = []
for i in range(len(data)):
    data_to_pd.append({'english_tokens': " ".join(data[i]['english_tokens']),
                       'russian_tokens': " ".join(data[i]['russian_tokens'])})

print(len(data_to_pd))

496059


In [5]:
df = pd.DataFrame(data_to_pd)
df.head()

,english_tokens,russian_tokens
0,go .,марш !
1,go .,иди .
2,go .,идите .
3,hi .,здравствуйте .
4,hi .,привет !


In [6]:
df.to_csv('df_ready.csv', index = False)

In [22]:
data = pd.read_csv('df_ready.csv')
data = data.sample(frac = 1, random_state = 42)
data = data[:100]
data.head()

,english_tokens,russian_tokens
152381,it seems you were right .,"похоже , ты была права ."
494486,"if there are no dogs in heaven , then when i d...","если в раю нет собак , то после смерти я хочу ..."
47240,i 'll wash the dog .,я искупаю собаку .
43221,do you have a car ?,у вас машина есть ?
213091,i 'm glad you 're here today .,"я рада , что вы сегодня здесь ."


### Загрузка модели Helsinki

In [23]:
model_name = 'Helsinki-NLP/opus-mt-en-ru'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate_text(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True)
    outputs = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return translated_text

c:\Users\oudel\anaconda3\envs\newenv\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


### Проверка

In [24]:
english_text = "Hello, how are you?"
translated_text = translate_text(english_text, model, tokenizer)
print(translated_text)

Привет, как дела?


# Перевод

In [25]:
data['translated_text'] = data['english_tokens'].map(lambda x: translate_text(x, model, tokenizer))

In [26]:
data.head()

,english_tokens,russian_tokens,translated_text
152381,it seems you were right .,"похоже , ты была права .","Похоже, ты был прав."
494486,"if there are no dogs in heaven , then when i d...","если в раю нет собак , то после смерти я хочу ...","Если на небесах нет собак, тогда, когда я умру..."
47240,i 'll wash the dog .,я искупаю собаку .,Я помою собаку.
43221,do you have a car ?,у вас машина есть ?,У тебя есть машина?
213091,i 'm glad you 're here today .,"я рада , что вы сегодня здесь .","Я рад, что ты сегодня здесь."
